In [1]:
from gensim.models import FastText
from gensim.models.fasttext import load_facebook_model
import gensim.downloader as downloader
import pickle
import os
import pandas as pd
from multiprocessing import Pool, cpu_count

In [3]:
model = load_facebook_model('/data/fasttext_models/pretrained/wiki.en.bin')

/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
type(model)

gensim.models.fasttext.FastText

In [6]:
model.workers = cpu_count() - 2

In [7]:
non_emar_df = pd.read_pickle('/data/progress_notes/non_emar_notes_tokenized.pickle')

In [8]:
non_emar_df.sample(5).T

,2069461,2441325,6173806,8776490,148991
client,avante,avante,avante,greystone,avante
FacilityID,7,8,26,33,1
PatientID,257254,293718,310313,1.00288e+07,162495
CreatedDate,2014-10-10 08:58:00.513000,2016-08-29 22:44:50.850000,2017-01-03 14:44:11.443000,2018-05-12 00:44:37.793000,2013-02-24 14:20:22.980000
ProgressNoteID,7296606,15801921,17248003,12060989,1264005
ProgressNoteType,* Skin / Wound Note,* General NURSING Note,* General NURSING Note,Health Status Note,* Skilled Nursing Note
Section,Plan,Note Text,Note Text,Note Text,Assessment
NoteText,,Resident in bed watching tv. Able to verbalize...,"LOA to appt this am & returned. Hgb 9.0, had i...",ADMISSION: Member received lying in bed. Is a...,Pt in bed in stable condition. Vital signs sta...
tokens,(),"(resident, in, bed, watching, tv, . , able, to...","(loa, to, appt, this, am, & , returned, . , h...","(admission, : , member, received, lying, in, b...","(pt, in, bed, in, stable, condition, . , vital..."


In [9]:
tokens = non_emar_df.loc[non_emar_df['tokens'].apply(len)>2, 'tokens'].to_list()

In [10]:
len(tokens)

8393856

In [11]:
%%time
model.build_vocab(tokens, update=True)

CPU times: user 3min 55s, sys: 11.3 s, total: 4min 6s
Wall time: 4min 6s


In [ ]:
%%time
model.train(sentences=tokens, total_examples=len(tokens), window=7, epochs=20)

In [ ]:
model.save('/data/models/ft_non_emar.model')

In [ ]:
!aws s3 cp /data/models/ft_non_emar.model s3://saiva-dev-data-bucket/data/fasttext_models/2019-08-29/ft_non_emar.model

In [ ]:
!aws s3 sync /data/models/ s3://saiva-dev-data-bucket/data/fasttext_models/2019-08-29/

In [50]:
emar_df = pd.read_pickle('/data/emar_notes_tokenized.pickle')
tokens = emar_df.loc[emar_df['tokens'].apply(len)>2, 'tokens'].to_list()

In [51]:
model = FastText(size=300, window=5, workers=os.cpu_count()-2, min_count=50)

In [52]:
model.wv = starting_vectors

In [ ]:
%%time
model.build_vocab(tokens, update=True)

In [ ]:
%%time
model.train(sentences=tokens, total_examples=len(tokens), window=7, epochs=30)

In [ ]:
model.save('/data/models/ft_emar.model')

In [17]:
!aws s3 sync /data/models/ s3://saiva-dev-data-bucket/data/fasttext_models/2019-08-27/